In [5]:
import pandas as pd
from collections import Counter
import spacy,re,json
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from spacy import displacy  

from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

import re
from pprint import pprint
import os
import warnings
warnings.filterwarnings('ignore')

from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

# ============================================================
## 基于 Spacy 的 NER 模型

In [6]:
PATH ='data/file_ner.json1'

annotated_data = []
lines=[]
with open(PATH, 'r',encoding = 'utf-8 sig') as f:
    lines = f.readlines()

for line in lines:
    data = json.loads(line)
    text = data['text']
    entities = []
    for label in data['labels']:
        entities.append(label)
    annotated_data.append((text, {"entities": data['labels']}))

In [7]:
training_data=  annotated_data[0:200]
test_data = annotated_data[200:220]

In [8]:
print(len(training_data), len(test_data))

200 20


In [9]:
training_data[0]

('"Abhishek Jha Application Development Associate - Accenture  Bengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a  • To work for an organization which provides me the opportunity to improve my skills and knowledge for my individual and company\'s growth in best possible ways.  Willing to relocate to: Bangalore, Karnataka  WORK EXPERIENCE  Application Development Associate  Accenture -  November 2017 to Present  Role: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries for the Bot which will be triggered based on given input. Also, Training the bot for different possible utterances (Both positive and negative), which will be given as input by the user.  EDUCATION  B.E in Information science and engineering  B.v.b college of engineering and technology -  Hubli, Karnataka  August 2013 to June 2017  12th in Mathematics  Woodbine modern school  April 2011 to March 2013  10th  Kendriya Vidyalaya  April 2001 to March 2011  SKILLS  C (

In [10]:
def main(model=None, output_dir=None, n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in training_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(training_data)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(training_data, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

    # test the trained model
    for text, _ in training_data:
        doc = nlp(text)
#         print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
#         print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in training_data:
            doc = nlp2(text)
#             print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
#             print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

# trained this model with 100 iterations in google colab using GPU runtime with loss: Losses {'ner': 465.24571515528373} 
main(output_dir='ner_model/') 

Created blank 'en' model
Losses {'ner': 16768.48688307873}
Losses {'ner': 2761.7945596546524}
Losses {'ner': 2586.147553201028}
Losses {'ner': 2763.2860978442395}
Losses {'ner': 2439.4767436901693}
Losses {'ner': 2687.634682729234}
Losses {'ner': 2355.694131632732}
Losses {'ner': 2189.2004296167097}
Losses {'ner': 2085.9152812762695}
Losses {'ner': 1990.533282201601}
Losses {'ner': 1912.0705477855574}
Losses {'ner': 1851.9641642381605}
Losses {'ner': 1800.518135927293}
Losses {'ner': 1707.7367110706346}
Losses {'ner': 1723.845531718316}
Losses {'ner': 1701.1866226226925}
Losses {'ner': 1561.5885531001795}
Losses {'ner': 1501.297711165509}
Losses {'ner': 1604.1706923918568}
Losses {'ner': 1385.7271474510703}
Losses {'ner': 1496.3298702131035}
Losses {'ner': 1346.2760019554257}
Losses {'ner': 1429.2420476255181}
Losses {'ner': 1287.5133737691833}
Losses {'ner': 1438.9120273613385}
Losses {'ner': 1145.246637270898}
Losses {'ner': 1284.8927977882884}
Losses {'ner': 1152.3211780370452}
Loss

# ==========================================================
## 导入 NER 模型，识别 JD 中的所有技能类关键词

In [14]:
ner_model = spacy.load('ner_model/')

In [15]:
skill_set = {}
link_set = {}

In [16]:
ind = 0

for root,dirs,files in os.walk(r"E:\dataset\dataset"):
    for file in files:
        ind += 1
        
        try:
            #获取文件路径
            my_file = open( os.path.join(root,file),encoding = 'utf-8' )
            my_string = my_file.readlines()

            for string in my_string[1:]:
                doc = ner_model( string )

                temp_list = []
                for chunk in doc.ents:
                    temp_list.append( chunk.text )
                    
                temp_set = set( temp_list )
                
                for tem in list(temp_set):
                    if tem not in skill_set.keys():
                        print( tem )
                        
                    if skill_set.get( tem, 0 ) == 0:
                        skill_set[tem] = 1
                    else:
                        skill_set[tem] += 1
                        
                for t1 in list(temp_set):
                    for t2 in list(temp_set):
                        if t1.text != t2.text:
                            if link_set.get( t1+'|'+t2, 0 ) == 0:
                                link_set[t1+'|'+t2] = 1
                            else:
                                link_set[t1+'|'+t2] += 1
                            

            my_file.close()
        
            if ind % 100 == 0:
                print( '------------------------------------------------------------------------',ind )
                
#             if ind > 2000:
#                 break
                
        except:
            pass

Logistic Regression
Python libraries
Machine Learning
Python
BlackRock
R
Java
Map/Reduce
C
C++
SQL
artificial neural networks
Random Forest
Spark
SAS
MATLAB
Neural Networks
Operations Research
software prototyping
Deep Learning
via SQL
C ##
C#
Javascript
Analytics
Consumer experience
Hadoop
TensorFlow
TV planning
To
Data Analytics
Deep Neural Networks
Google Analytics
Google Big Query (a
Create cutting
·
machine learning
MLlib
Python R
Data Engineers
Google Platform
containerisation (Kubernetes
Kubeflow
Cloud Enablement
Infrastructure Support
; Testing
Docker
Data Engineering
Business Intelligence
Informatica
SAP HANA
Node.JS
Excel
e.g. Python
data mining/predictive analytics
as Redshift
S3
Athena
Kinesis
SPSS
Tensorflow
Development
Engineering Support
Matlab
Powerpoint
Microsoft Office
Visio
predictive modeling
problem-solving
NoSQL
regression analysis
visual (presentations
non-parametric
national origin
Oracle
create insightful
XML
MS Windows
Visualization
Oracle/SQL
GitHub
- SQL
MA 

Sciences
------------------------------------------------------------------------ 1700
GCP and/or
Azure IOT
Adobe Advertising Cloud
Bigquery
git
Unix/
administrative/office
model building
nature language
Access
SCAN

Tableau R
and/or NoSQL
databases
Optimization Predictive
product management
SPSS and/or
Marketing Analytics
Operations
Product Management
Application Development
Python(PyPi),Text analysis
Microsoft Reporting Studio
------------------------------------------------------------------------ 1800
ECommerce
Random Forests
Deterministic Forests
Search Engine Marketing
Facebook Analytics
and/or Amazon Marketing
Data recruitment
Design Quality
SASUnix
Director
Unix
NOSQL databases
MSSQL   
Jira
DevOps   
PERKS
Python/PySpark
reinforcement learning
Azure Kubernetes
Azure Databricks
Text Analytics
Neon
SystemML
Data Intelligence
Data Governance
feature transformation
non-experts
Python  
Spark and/or
R SQL Experience
Problem Solving
Core Attributes
Judgment/
Statistical Analysis
SQL

Gaussian process
machine intelligence
IBM Analytics
Data Processing
Azure Experience
As a System
Network analysis
Regression analysis
Software
Geospatial Analytics
advanced statistical analysis
team player
consumer marketing
sales
A B.S.
specifically R
Java script
NLP &amp
Crop/Plant
Windows 2003
age
six
Software Engineer
Business Intelligence Engineer
Data Architect building
AWS AI
Google AI
Medication Adherence
------------------------------------------------------------------------ 5000
NLTK

Michigan - 150k Location
AWZ
Spark/PySpark
testing
database querying
R & SQL
RDS
Scala
anomaly detection
Bash Scripting
Apache SOLR
statistical software (R
Comcast PABS
Prescriptive Analytics
MATLAB)Experience extracting
Builds predictive and/or
TX 75014 

Menlo Ventures
Postgis
PostgresSQL
PgAdmin
forecasting/predictive analytics
strong SQL experience
Gradient Boosting
Qlikview Data visualization
Manufacturing Analytics
MRP
stay abreast
Enlighten IT Consulting
javascript
css
as SQL
Math or sim

In [10]:
# skill_set = dict( Counter(skill_set).most_common(300) )
# skill_set

# ============================================================
## 基于句法分析抽取层级关系

### 根据每个技能类关键词，向上找最小分支树，判断依存关系

In [17]:
nlp = spacy.load("en_core_web_sm")

In [18]:
def is_father( word, root ):
    if word == root:
        return True
    
    while word != word.head:
        word = word.head
        if word == root:
            return True
    
    return False

In [22]:
ind = 0

total_link_list = []
for root,dirs,files in os.walk(r"E:\dataset\dataset"):
    for file in files:
        ind += 1
#         if ind < 500:
#             continue
            
        if ind % 2 == 1:
            continue
        
        my_file = open( os.path.join(root,file),encoding = 'utf-8' )

        my_string = my_file.readlines()
        for string_list in my_string[1:]:
            string_list = string_list.replace('e.g.','like')
            string_list = string_list.replace('.)',')')
#             for string in [sentence for sentence in re.split(r'[？?！!。；;：:\n\r·]', string_list) if sentence]:
            for string in re.split(r'[？?！!。.；;：:\n\r·•*]', string_list):
#                 try:
                doc = nlp( string )

                my_chunks_list = []
                roor_chunks_dict = {}

                for chunk in doc.noun_chunks:
                    if chunk.text in skill_set.keys():
                        my_chunks_list.append( chunk.root )
                        roor_chunks_dict[ str(chunk.root)] = str(chunk)

                if len( my_chunks_list ) > 1:
#                     print( 'skill word:', my_chunks_list )
                    Total_List = []

                    for word in my_chunks_list:
                        temp_list = []
                        temp_list.append(word)

                        pre = word
                        while word != word.head:
                            pre = word
                            relation = word.dep_
                            word = word.head

                            if 'N' in word.tag_ and (relation != 'conj' and relation != 'appos'):
                                if word in my_chunks_list:
                                    temp_list.append( word )

                                    if len( [str( roor_chunks_dict[str(w)] ) for w in temp_list if 'xperience' not in str(w)] ) == 2:
                                            print(string)
                                            print('result1:') 
                                            print( '-->'.join([str( roor_chunks_dict[str(w)] ) for w in temp_list]) )
                                            total_link_list.append( [str( roor_chunks_dict[str(w)] ) for w in temp_list] )
                                            print()

                                else:
                                    tree_list = []
                                    for t in my_chunks_list:
                                        if t.dep_ == 'compound' and is_father( t, word ) == True and is_father( t, pre ) == False:
                                            tree_list.append( t )

                                    if len(tree_list) == 1:
                                        key1 = roor_chunks_dict[ str(temp_list[0]) ]
                                        key2 = roor_chunks_dict[ str(tree_list[0]) ]
                                        
                                        if 'xperience' not in key1 and 'xperience' not in key2:
                                            print(string)
                                            print('result2:')  
                                            
                                            print( '-->'.join( [ key1, key2 ] ) )
                                            total_link_list.append( [ key1, key2 ] )
                                            print()
                                break
                
        my_file.close()

        if ind % 500 == 0:
            print( '==================================================',ind )
            
        if ind > 2000:
            break

Technology We Use
result1:
We-->Technology

 You will develop models using algorithms for pattern detection and optimization
result1:
algorithms-->models

Research in cryptomarket microstructure, including the fine details and mechanics of crypto exchanges, and their business
result1:
mechanics-->Research

 Create visualizations in support of data science projects from beginning to end, including understanding the business need, understanding the analysis findings, prototyping models, and deploying completed models to deliver business impact to the organization, in collaboration with experienced data scientists
result1:
models-->prototyping

- Experience developing software with C++, C, Java, Python, R, or Matlab, software applications in Linux, UNIX, Windows environments, data analysis algorithms, data management approaches, relational databases, or machine learning algorithms
result1:
algorithms-->machine learning

   Tools such as Python (including Pandas, NumPy and Scikit-learn) an

Conducts advanced analytics leveraging predictive modeling, machine learning, simulation, optimization and other techniques to deliver insights or develop analytical solutions to achieve business objectives
result1:
predictive modeling-->advanced analytics

Conducts advanced analytics leveraging predictive modeling, machine learning, simulation, optimization and other techniques to deliver insights or develop analytical solutions to achieve business objectives
result1:
machine learning-->advanced analytics

Conducts advanced analytics leveraging predictive modeling, machine learning, simulation, optimization and other techniques to deliver insights or develop analytical solutions to achieve business objectives
result1:
simulation-->advanced analytics

Conducts advanced analytics leveraging predictive modeling, machine learning, simulation, optimization and other techniques to deliver insights or develop analytical solutions to achieve business objectives
result1:
optimization-->advance

Advanced-level skills with relational databases, including SQL and utilizing data stored in complex schemas
result1:
SQL-->relational databases

Design and Implementation of refinery data product using Spark, Map-Reduce, Hive, HDFS, Spring, Hibernate and Java
result1:
Spark-->Design

Design and implementation of data product on Hadoop using Titan, Tinker-pop, HBase, Gremlin and Java
result1:
Java-->Design

Knowledge of statistics including hypothesis testing
result1:
hypothesis testing-->statistics

Statistical analysis, using SAS, R, Python or equivalent
result1:
SAS-->Statistical analysis

Statistical analysis, using SAS, R, Python or equivalent
result1:
R-->Statistical analysis

Statistical analysis, using SAS, R, Python or equivalent
result1:
Python-->Statistical analysis

-Understanding of predictive analytics techniques leveraging machine learning algorithms 
result1:
algorithms-->machine learning

 Candidates should be fluent in Python for analytics including, Pandas, Numpy, Sci

Conducts advanced analytics leveraging predictive modeling, machine learning, simulation, optimization and other techniques to deliver insights or develop analytical solutions to achieve business objectives
result1:
predictive modeling-->advanced analytics

Conducts advanced analytics leveraging predictive modeling, machine learning, simulation, optimization and other techniques to deliver insights or develop analytical solutions to achieve business objectives
result1:
machine learning-->advanced analytics

Conducts advanced analytics leveraging predictive modeling, machine learning, simulation, optimization and other techniques to deliver insights or develop analytical solutions to achieve business objectives
result1:
simulation-->advanced analytics

Conducts advanced analytics leveraging predictive modeling, machine learning, simulation, optimization and other techniques to deliver insights or develop analytical solutions to achieve business objectives
result1:
optimization-->advance

================================================== 2000


In [ ]:
ind = 0

total_link_list = []
for root,dirs,files in os.walk(r"E:\dataset\dataset"):
    for file in files:
        ind += 1
        
        my_file = open( os.path.join(root,file),encoding = 'utf-8' )

        my_string = my_file.readlines()
        for string_list in my_string[1:]:
            string_list = string_list.replace('e.g.','like')
            string_list = string_list.replace('.)',')')
#             for string in [sentence for sentence in re.split(r'[？?！!。；;：:\n\r·]', string_list) if sentence]:
            for string in re.split(r'[ -？?！!。.；;：:\n\r·•*]', string_list):
#                 try:
                doc = nlp( string )

                my_chunks_list = []
                roor_chunks_dict = {}

                for chunk in doc.noun_chunks:
                    if chunk.text in skill_set.keys():
                        my_chunks_list.append( chunk.root )
                        roor_chunks_dict[ str(chunk.root)] = str(chunk)

                if len( my_chunks_list ) > 1:
#                     print( 'skill word:', my_chunks_list )
                    Total_List = []

                    for word in my_chunks_list:
                        temp_list = []
                        temp_list.append(word)

                        pre = word
                        while word != word.head:
                            pre = word
                            relation = word.dep_
                            word = word.head

                            if 'N' in word.tag_ and (relation != 'conj' and relation != 'appos'):
                                if word in my_chunks_list:
                                    temp_list.append( word )

                                    if len( [str( roor_chunks_dict[str(w)] ) for w in temp_list if 'xperience' not in str(w)] ) == 2:
                                            print( '-->'.join([str( roor_chunks_dict[str(w)] ) for w in temp_list]) )
                                            total_link_list.append( [str( roor_chunks_dict[str(w)] ) for w in temp_list] )

                                else:
                                    tree_list = []
                                    for t in my_chunks_list:
                                        if t.dep_ == 'compound' and is_father( t, word ) == True and is_father( t, pre ) == False:
                                            tree_list.append( t )

                                    if len(tree_list) == 1:
                                        key1 = roor_chunks_dict[ str(temp_list[0]) ]
                                        key2 = roor_chunks_dict[ str(tree_list[0]) ]
                                        
                                        if 'xperience' not in key1 and 'xperience' not in key2:
                                            print( '-->'.join( [ key1, key2 ] ) )
                                            total_link_list.append( [ key1, key2 ] )
                                break
                                    
        my_file.close()

        if ind % 500 == 0:
            print( '==================================================',ind )

In [19]:
link_df = pd.DataFrame( total_link_list )
link_df = link_df.rename(columns = {0:'node1',1:'node2'})
link_df

,node1,node2
0,Tableau,data visualization
1,algorithms,Deep learning
2,Tableau,data visualization
3,algorithms,models
4,business,advanced algorithms
...,...,...
898,algorithms,deep learning
899,algorithms,machine learning
900,algorithms,machine learning
901,algorithms,machine learning


# 关系绘制

In [22]:
from unittest.mock import patch
from nose.tools import assert_equal, assert_in
from pyecharts import options as opts
from pyecharts.charts.basic_charts.graph import Graph

from pyecharts import options as opts
from pyecharts.charts import Graph

nodes = []
links = []

nodes_set = []

for ind, row in link_df.iterrows():
    
    if row['node1'] != row['node2']:
        
        if row['node1'] not in nodes_set:
            nodes_set.append( row['node1'] )
            nodes.append( {'name': row['node1'] , "symbolSize": 15 + skill_set[row['node1']] / 20 } )
            
        if row['node2'] not in nodes_set:
            nodes_set.append( row['node2'] )
            nodes.append( {'name': row['node2'] , "symbolSize": 15 + skill_set[row['node2']] / 20 } )
        
        links.append({"source": row['node2'], "target": row['node1']})

Graph().add(
    "", 
    nodes, 
    links, 
    repulsion=8000,
#     linestyle_opts=opts.LineStyleOpts(width=2, curve=0.1, opacity=0.9),
    linestyle_opts=opts.LineStyleOpts(width=0.5, curve=0.3, opacity=0.7),
    label_opts=opts.LabelOpts( ),
    categories=None,
    is_focusnode=True,
#     layout="none",
    is_roam=True
    
).set_global_opts(title_opts=opts.TitleOpts(title="")).render_notebook()